In [41]:
from collections import defaultdict, Counter
corpus = ['low'] * 5 + ['lower'] * 2 + ['newest'] * 6 + ['widest'] * 3

set_corpus = set([i for i in corpus])

dictionary = defaultdict(int)
for i in corpus:
    dictionary[" ".join([k for k in i]) + " _"] += 1
print(dictionary)

defaultdict(<class 'int'>, {'l o w _': 5, 'l o w e r _': 2, 'n e w e s t _': 6, 'w i d e s t _': 3})


In [8]:
list(zip(corpus, ["_" for _ in corpus]))

[('abc', '_'), ('aba', '_')]

In [6]:
for a,b in zip(corpus, ["_" for _ in corpus]):
    print(a, b)

abc _
aba _


In [10]:
p = "AABBAAAABB"
p.count("AA")

3

In [11]:
from IPython.display import display, Markdown, Latex
display(Markdown('*some markdown* $\phi$'))

*some markdown* $\phi$

In [38]:
import re, collections
from IPython.display import display, Markdown, Latex
max_vocab_size = 100
num_merges = max_vocab_size - len(set_corpus) - 6

def get_stats(dictionary):
    # 유니그램의 pair들의 빈도수를 카운트
    pairs = collections.defaultdict(int)
    for word, freq in dictionary.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
#     print('현재 pair들의 빈도수 :', dict(pairs))
    return pairs

def merge_dictionary(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

bpe_codes = {}
bpe_codes_reverse = {}

for i in range(num_merges):
    display(Markdown("### Iteration {}".format(i + 1)))
    pairs = get_stats(dictionary)
    best = max(pairs, key=pairs.get)
    dictionary = merge_dictionary(best, dictionary)

    bpe_codes[best] = i
    bpe_codes_reverse[best[0] + best[1]] = best

for i in bpe_codes_reverse:
    set_corpus.add(i)
id2word.append(sorted(set_corpus, key = lambda x : len(x)))

### Iteration 1

### Iteration 2

### Iteration 3

### Iteration 4

### Iteration 5

...

### Iteration 13

### Iteration 14

### Iteration 15

### Iteration 16

ValueError: max() arg is an empty sequence

In [27]:
print(bpe_codes)

{('e', 's'): 0, ('es', 't'): 1, ('est', '_'): 2, ('l', 'o'): 3, ('lo', 'w'): 4, ('n', 'e'): 5, ('ne', 'w'): 6, ('new', 'est_'): 7, ('low', '_'): 8, ('w', 'i'): 9}


In [19]:
" ".join([k for k in "AB"])

'A B'

In [28]:
bpe_codes_reverse

{'es': ('e', 's'),
 'est': ('es', 't'),
 'est_': ('est', '_'),
 'lo': ('l', 'o'),
 'low': ('lo', 'w'),
 'ne': ('n', 'e'),
 'new': ('ne', 'w'),
 'newest_': ('new', 'est_'),
 'low_': ('low', '_'),
 'wi': ('w', 'i')}

In [81]:
from typing import List, Dict, Set
from itertools import chain
import re
from collections import defaultdict, Counter


def get_stats(dictionary):
    # 유니그램의 pair들의 빈도수를 카운트
    pairs = defaultdict(int)
    for word, freq in dictionary.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
#     print('현재 pair들의 빈도수 :', dict(pairs))
    return pairs

def merge_dictionary(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out



def build_bpe(
        corpus: List[str],
        max_vocab_size: int
) -> List[int]:
    """ BPE Vocabulary Builder
    Implement vocabulary builder for byte pair encoding.
    Please sort your idx2word by subword length in descending manner.

    Hint: Counter in collection library would be helpful

    Note: If you convert sentences list to word frequence dictionary,
          building speed is enhanced significantly because duplicated words are
          preprocessed together

    Arguments:
    corpus -- List of words to build vocab
    max_vocab_size -- The maximum size of vocab

    Return:
    idx2word -- Subword list
    """
    # Special tokens
    PAD = BytePairEncoding.PAD_token  # Index of <PAD> must be 0
    UNK = BytePairEncoding.UNK_token  # Index of <UNK> must be 1
    CLS = BytePairEncoding.CLS_token  # Index of <CLS> must be 2
    SEP = BytePairEncoding.SEP_token  # Index of <SEP> must be 3
    MSK = BytePairEncoding.MSK_token  # Index of <MSK> must be 4
    SPECIAL = [PAD, UNK, CLS, SEP, MSK]

    WORD_END = BytePairEncoding.WORD_END  # Use this token as the end of a word
    # YOUR CODE HERE
    set_corpus = set()
    for i in corpus:
        for j in i:
            set_corpus.add(j)
    set_corpus.add(WORD_END)
    dictionary = defaultdict(int)
    for i in corpus:
        dictionary[" ".join([k for k in i]) + " _"] += 1

    idx2word = [PAD, UNK, CLS, SEP, MSK]

    bpe_codes = {}
    bpe_codes_reverse = {}

    num_merges = max_vocab_size - len(set_corpus) - 5
#     num_merges = max_vocab_size

    for i in range(num_merges):
        pairs = get_stats(dictionary)
        if len(pairs) == 0:
            break
        best = max(pairs, key=pairs.get)
        dictionary = merge_dictionary(best, dictionary)

        bpe_codes[best] = i
        bpe_codes_reverse[best[0] + best[1]] = best

    for i in bpe_codes_reverse:
        set_corpus.add(i)
    idx2word += sorted(set_corpus, key = lambda x : -len(x))
    
    return idx2word

In [82]:
class BytePairEncoding(object):
    """ Byte Pair Encoding class
    We aren't gonna use this class for encoding. Because it is too slow......
    We will use sentence piece Google have made.
    Thus, this class is just for special token index reference.
    """
    PAD_token = '<pad>'
    PAD_token_idx = 0
    UNK_token = '<unk>'
    UNK_token_idx = 1
    CLS_token = '<cls>'
    CLS_token_idx = 2
    SEP_token = '<sep>'
    SEP_token_idx = 3
    MSK_token = '<msk>'
    MSK_token_idx = 4

    WORD_END = '_'

    def __init__(self, corpus: List[List[str]], max_vocab_size: int) -> None:
        self.idx2word = build_bpe(corpus, max_vocab_size)

    def encode(self, sentence: List[str]) -> List[int]:
        return encode(sentence, self.idx2word)

    def decoder(self, tokens: List[int]) -> List[str]:
        return decode(tokens, self.idx2word)
    
    
PAD = BytePairEncoding.PAD_token
UNK = BytePairEncoding.UNK_token
CLS = BytePairEncoding.CLS_token
SEP = BytePairEncoding.SEP_token
MSK = BytePairEncoding.MSK_token
WORD_END = BytePairEncoding.WORD_END
corpus = ['low'] * 5 + ['lower'] * 2 + ['newest'] * 6 + ['widest'] * 3
vocab = set(build_bpe(corpus, max_vocab_size=24))
vocab

{'<cls>',
 '<msk>',
 '<pad>',
 '<sep>',
 '<unk>',
 '_',
 'd',
 'e',
 'es',
 'est',
 'est_',
 'i',
 'l',
 'lo',
 'low',
 'n',
 'ne',
 'new',
 'newest_',
 'o',
 'r',
 's',
 't',
 'w'}

In [39]:
from collections import defaultdict, Counter
corpus = ['low'] * 5 + ['lower'] * 2 + ['newest'] * 6 + ['widest'] * 3

set_corpus = set([i for i in corpus])

dictionary = defaultdict(int)
for i in corpus:
    dictionary[" ".join([k for k in i]) + " _"] += 1
print(dictionary)

defaultdict(<class 'int'>, {'l o w _': 5, 'l o w e r _': 2, 'n e w e s t _': 6, 'w i d e s t _': 3})


In [40]:
import re, collections
from IPython.display import display, Markdown, Latex

def get_stats(dictionary):
    # 유니그램의 pair들의 빈도수를 카운트
    pairs = collections.defaultdict(int)
    for word, freq in dictionary.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_dictionary(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

bpe_codes = {}
bpe_codes_reverse = {}

max_vocab_size = 100
num_merges = max_vocab_size - len(set_corpus) - 6

for i in range(num_merges):
    pairs = get_stats(dictionary)
    best = max(pairs, key=pairs.get)
    dictionary = merge_dictionary(best, dictionary)

    bpe_codes[best] = i
    bpe_codes_reverse[best[0] + best[1]] = best

    print("new merge: {}".format(best))
    print("dictionary: {}".format(dictionary))

### Iteration 1

현재 pair들의 빈도수 : {('l', 'o'): 7, ('o', 'w'): 7, ('w', '_'): 5, ('w', 'e'): 8, ('e', 'r'): 2, ('r', '_'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('e', 's'): 9, ('s', 't'): 9, ('t', '_'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3}
new merge: ('e', 's')
dictionary: {'l o w _': 5, 'l o w e r _': 2, 'n e w es t _': 6, 'w i d es t _': 3}


### Iteration 2

현재 pair들의 빈도수 : {('l', 'o'): 7, ('o', 'w'): 7, ('w', '_'): 5, ('w', 'e'): 2, ('e', 'r'): 2, ('r', '_'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'es'): 6, ('es', 't'): 9, ('t', '_'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'es'): 3}
new merge: ('es', 't')
dictionary: {'l o w _': 5, 'l o w e r _': 2, 'n e w est _': 6, 'w i d est _': 3}


### Iteration 3

...

### Iteration 15

현재 pair들의 빈도수 : {('lower', '_'): 2}
new merge: ('lower', '_')
dictionary: {'low_': 5, 'lower_': 2, 'newest_': 6, 'widest_': 3}


### Iteration 16

현재 pair들의 빈도수 : {}


ValueError: max() arg is an empty sequence

In [61]:
set_corpus = set([i for i in corpus])
set_corpus

{'low', 'lower', 'newest', 'widest'}